In [1]:
import pandas as pd

In [22]:
source = "TATA"
destination = "ALD"

In [3]:
stations = pd.read_csv("datasets/stations.csv")
dict = {}
dict1 = {}
for i in range(0,8359):
    try:
        if len(stations["Latitude"].iloc(0)[i]) == 10:
            lat = stations["Latitude"].iloc(0)[i][:2] 
            lat += stations["Latitude"].iloc(0)[i][3:7]
        else:
            lat = "0"
            lat += stations["Latitude"].iloc(0)[i][:1] 
            lat += stations["Latitude"].iloc(0)[i][2:6]
        long = stations["Longitude"].iloc(0)[i][:2] 
        long += stations["Longitude"].iloc(0)[i][3:7]
        dict1[stations["Station Codes"].iloc(0)[i]] = lat + long
    except:
        pass
    dict[stations["Station Codes"].iloc(0)[i]] = i

In [4]:
trains = pd.read_csv("datasets/trains.csv")
trains = trains[trains["Available Classes"] != "UNRESERVED"]
trains = trains.dropna()

In [5]:
running_days = []
for days in trains["Running Days"]:
    if "Daily" in days:
        running_days.append("1111111")
        continue
    binary_string = ""
    
    if "SUN" in days:
        binary_string += '1'
    else:
        binary_string += '0'
    
    if "MON" in days:
        binary_string += '1'
    else:
        binary_string += '0'
    
    if "TUE" in days:
        binary_string += '1'
    else:
        binary_string += '0'
    
    if "WED" in days:
        binary_string += '1'
    else:
        binary_string += '0'
    
    if "THU" in days:
        binary_string += '1'
    else:
        binary_string += '0'
    
    if "FRI" in days:
        binary_string += '1'
    else:
        binary_string += '0'
    
    if "SAT" in days:
        binary_string += '1'
    else:
        binary_string += '0'
    
    running_days.append(binary_string)

In [17]:
matrix = []
for i in range(0,8359):
    matrix.append([])
for i in range(0,2449):
    train_number = trains["Train Number"].iloc(0)[i]
    station_string = trains["Route with codes"].iloc(0)[i]
    station = station_string.split('$')
    station = station[:-1]
    arrival_string = trains["Arrival Times"].iloc(0)[i]
    arrival = arrival_string.split('$')
    arrival = arrival[:-1]
    departure_string = trains["Departure Times"].iloc(0)[i]
    departure = departure_string.split('$')
    departure = departure[:-1]
    right_string = trains["Right Time (0-15 mins)"].iloc(0)[i]
    right = right_string.split('$')
    right = right[:-1]
    slight_string = trains["Slight Delay (15-60 mins)"].iloc(0)[i]
    slight = slight_string.split('$')
    slight = slight[:-1]
    significant_string = trains["Significant Delay(>1 Hour)"].iloc(0)[i]
    significant = significant_string.split('$')
    significant = significant[:-1]
    
    delay_array = []
    length = len(station)
    
    for j in range(0,length):
        try:
            delay = float(right[j]) * 5 + float(slight[j]) * 40 + float(significant[j]) * 90
            delay_array.append(delay)    
        except:
            delay_array.append(delay_array[j-1])
            
    for j in range(0, length - 1):
        source_station = station[j]
        source_hour = departure[j][:2]
        source_minute = departure[j][3:5]
        source_day = departure[j][11:12]
        try:
            source_departure = int(source_day) * 24 * 60 + int(source_hour) * 60 + int(source_minute) - 24 * 60
        except:
            continue
        source_departure %= (7 * 24 * 60)
        source_delay = delay_array[j]
        
        index = dict[source_station]
        
        for k in range(j + 1, length):
            destination_station = station[k]
            destination_hour = arrival[k][:2]
            destination_minute = arrival[k][3:5]
            destination_day = arrival[k][11:12]
            destination_arrival = int(destination_day) * 24 * 60 + int(destination_hour) * 60 + int(destination_minute) - 24 * 60
            destination_arrival %= (7 * 24 * 60)
            destination_delay = delay_array[k]
            
            for l in range(0,7):
                if running_days[i][l] == '0':
                    continue
                    
                departure_time = l * 24 * 60 + source_departure
                departure_time %= (24 * 7 * 60)
                arrival_time = l * 24 * 60 + destination_arrival
                arrival_time %= (24 * 7 * 60)
                
                structure = {}
                structure["source_station"] = source_station
                structure["source_departure"] = departure_time
                structure["source_delay"] = source_delay
                structure["destination_station"] = destination_station
                structure["destination_arrival"] = arrival_time
                structure["destination_delay"] = destination_delay
                structure["train_number"] = train_number
                
                matrix[index].append(structure)

In [24]:
class QueueData:
    starting_time = 0
    ending_time = 0
    last_train = 0
    last_station = ""
    train_list = []
    station_list = []
    intermissions = 0

In [21]:
routes = []

In [35]:
source_index = dict[source]
destination_index = dict[destination]
queue = []
i = 0

for data in matrix[source_index]:
    queue_data = QueueData()
    queue_data.starting_time = data["source_departure"]
    queue_data.ending_time = data["destination_arrival"]
    queue_data.last_train = data["train_number"]
    queue_data.train_list.append(queue_data.last_train)
    queue_data.last_station = data["destination_station"]
    queue_data.station_list.append(source)
    queue_data.station_list.append(queue_data.last_station)
    queue_data.intermissions =  queue_data.intermissions + 1
    queue.append(queue_data)
    
while len(queue) != 0:
    queue_data = queue[0]
    queue.pop(0)
    if queue_data.intermissions == 2:
        continue
    station_index = dict[queue_data.last_station]
    arrival_time = queue_data.ending_time
    for data in matrix[station_index]:
        departure_time = data["source_departure"]
        if arrival_time > departure_time:
            departure_time += (7 * 24 * 60)
        if departure_time - arrival_time > 8 * 60:
            continue
        departure_time %= (7 * 24 * 60)
        queue_data.ending_time = data["destination_arrival"]
        queue_data.last_train = data["train_number"]
        queue_data.train_list.append(queue_data.last_train)
        queue_data.last_station = data["destination_station"]
        queue_data.station_list.append(queue_data.last_station)
        if queue_data.last_station == destination:
            routes.append(queue_data)
            continue
        queue_data.intermissions =  queue_data.intermissions + 1
        queue.append(queue_data)

KeyboardInterrupt: 